In [1]:
# ==================================================================================================== Step 1: Import Libraries and Load Images
import cv2
import numpy as np

# Set the path to the images
img = [
    "img/no (1).jpg",
    "img/no (2).jpg",
    "img/no (3).jpg",
    "img/no (4).jpg",
    "img/no (5).jpg",
    "img/no (6).jpg",
    "img/no (7).jpg",
    "img/no (8).jpg",
    "img/no (9).jpg",
    "img/no (10).jpg",
    "img/gg (1).jpg",
    "img/gg (2).jpg",
    "img/gg (3).jpg",
    "img/gg (4).jpg",
    "img/m (1).jpg",
    "img/m (2).jpg",
    "img/m (3).jpg",
    "img/m (4).jpg",
    "img/m (5).jpg",
    "img/m (6).jpg",
]

# Shuffle the images
np.random.shuffle(img)

# Prepare the correct answers
correct_ans = []
for i in img:
    if "no" in i:
        correct_ans.append("No")
    else:
        correct_ans.append("Yes")

# Load the images in grayscale
df_img = []
for i in img:
    df_img.append(cv2.imread(i, cv2.IMREAD_GRAYSCALE))

In [2]:
# ==================================================================================================== Step 2: Preprocess the Images
# Crop image of unwanted padding
cropped_img = []
for img in df_img:
    # Threshold the image
    threshold = cv2.threshold(img.copy(), 40, 255, cv2.THRESH_OTSU)[1]
    # Find the contours
    contours, _ = cv2.findContours(threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # Get largest contour
    c = max(contours, key=cv2.contourArea)
    # Get extreme points of the contour
    extleft = tuple(c[c[:, :, 0].argmin()][0])
    extright = tuple(c[c[:, :, 0].argmax()][0])
    exttop = tuple(c[c[:, :, 1].argmin()][0])
    extbot = tuple(c[c[:, :, 1].argmax()][0])
    # Crop the image
    cropped_img.append(img[exttop[1] : extbot[1], extleft[0] : extright[0]])

# Show the cropped images
# for i in range(len(cropped_img)):
#     cv2.imshow("Cropped Image", cropped_img[i])
#     cv2.waitKey(0)
# cv2.destroyAllWindows()

In [3]:
# ==================================================================================================== Step 3: Extract Features and Classify
# 1. Threshold
threshold_img = []

# Parameters
t_val = 100

for img in cropped_img:
    _, tres = cv2.threshold(img.copy(), t_val, 255, cv2.THRESH_BINARY)
    threshold_img.append(tres)

# 2. Erode and Dilate
ed_img = []
k_size = 3
kernel = np.ones((k_size, k_size), np.uint8)
total_iterations = 1
e_iterations = 1
d_iterations = 1
for img in threshold_img:
    for i in range(total_iterations):
        erode = cv2.erode(img.copy(), kernel, iterations=e_iterations)
        dilate = cv2.dilate(erode, kernel, iterations=d_iterations)
    ed_img.append(dilate)

# 3. Finding Contours, Draw Contours, Extract features
contour_img = []
output_img = []
output_ans = []

# Font
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 0.5
font_color = (255, 255, 255)
font_thickness = 1
text_origin_pred = (10, 20)
text_origin_ans = (10, 50)
text_origin_accurate = (100, 20)

# Output Bar Size
bar_height = 60
bar_width = 150

# Save result img
save_result = True

# Parameters
t_min_area = 0.005
t_max_area = 0.4

t_rect_ratio = 2.2

t_box_pos = 0.15

t_white_1 = 0.3

t_white_2 = 0.25


def img_add_top_bar(img, height: int, width: int):

    orig_height, orig_width = img.shape

    # Calculate the new image dimensions, ensuring the top bar fits
    new_height = height + orig_height
    new_width = max(orig_width, width)

    # Create a new image with the expanded height and width
    new_img = np.zeros((new_height, new_width), dtype=np.uint8)

    # Place the original image below the top bar
    new_img[height:, :orig_width] = img

    return new_img


index = 0
for img in ed_img:
    # Get the index of the image
    original_img = cropped_img[index]
    correct_answer = correct_ans[index]
    # Filter 1: Overall White Pixel Percentage too High
    if np.sum(img == 255) / (img.shape[0] * img.shape[1]) > t_white_1:
        new_canvas = original_img.copy()
        new_canvas = img_add_top_bar(new_canvas, bar_height, bar_width)
        # Add labels
        cv2.putText(new_canvas, "Pred:No", text_origin_pred, font, font_scale, font_color, font_thickness, cv2.LINE_AA)
        cv2.putText(new_canvas, "Ans:" + correct_answer, text_origin_ans, font, font_scale, font_color, font_thickness, cv2.LINE_AA)

        if correct_answer == "No":
            cv2.putText(new_canvas, "Correct", text_origin_accurate, font, font_scale, font_color, font_thickness, cv2.LINE_AA)
        else:
            cv2.putText(new_canvas, "Wrong", text_origin_accurate, font, font_scale, font_color, font_thickness, cv2.LINE_AA)

        # Save the output
        output_img.append(new_canvas)
        output_ans.append("No")
        print("Image ", index + 1, ": Prediction: No | Answer: ", correct_answer)
        if save_result:
            cv2.imwrite("output/" + str(index + 1) + ".jpg", new_canvas)
        # Continue to the next image
        index += 1
        continue

    # Find the contours
    contours, _ = cv2.findContours(img, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    new_canvas = np.zeros_like(img)
    cv2.drawContours(new_canvas, contours, -1, (255, 255, 255), 1)
    contour_img.append(contours)

    # Filter 2: Base on contour properties
    box_count = 0
    new_canvas = original_img.copy()
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        min_area = t_min_area * img.shape[0] * img.shape[1]
        max_area = t_max_area * img.shape[0] * img.shape[1]
        area = w * h
        bound_center = (x + w / 2, y + h / 2)
        low_x = t_box_pos * img.shape[1]
        low_y = t_box_pos * img.shape[0]
        up_x = img.shape[1] - low_x
        up_y = img.shape[0] - low_y

        if min_area < area < max_area and w / h < t_rect_ratio and h / w < t_rect_ratio and (up_x > bound_center[0] > low_x) and (up_y > bound_center[1] > low_y) and (np.sum(img[y : y + h, x : x + w] == 255) / area) > t_white_2:
            cv2.rectangle(new_canvas, (x, y), (x + w, y + h), (255, 255, 255), 1)
            box_count += 1

    if box_count > 0:
        new_canvas = img_add_top_bar(new_canvas, bar_height, bar_width)
        # Add labels
        cv2.putText(new_canvas, "Pred:Yes", text_origin_pred, font, font_scale, font_color, font_thickness, cv2.LINE_AA)
        cv2.putText(new_canvas, "Ans:" + correct_answer, text_origin_ans, font, font_scale, font_color, font_thickness, cv2.LINE_AA)
        if correct_answer == "No":
            cv2.putText(new_canvas, "Wrong", text_origin_accurate, font, font_scale, font_color, font_thickness, cv2.LINE_AA)
        else:
            cv2.putText(new_canvas, "Correct", text_origin_accurate, font, font_scale, font_color, font_thickness, cv2.LINE_AA)
        # Save the output
        output_img.append(new_canvas)
        output_ans.append("Yes")
        print("Image ", index + 1, ": Prediction: Yes | Answer: ", correct_answer)
        if save_result:
            cv2.imwrite("output/" + str(index + 1) + ".jpg", new_canvas)
    else:
        new_canvas = img_add_top_bar(new_canvas, bar_height, bar_width)
        # Add labels
        cv2.putText(new_canvas, "Pred:No", text_origin_pred, font, font_scale, font_color, font_thickness, cv2.LINE_AA)
        cv2.putText(new_canvas, "Ans:" + correct_answer, text_origin_ans, font, font_scale, font_color, font_thickness, cv2.LINE_AA)
        if correct_answer == "No":
            cv2.putText(new_canvas, "Correct", text_origin_accurate, font, font_scale, font_color, font_thickness, cv2.LINE_AA)
        else:
            cv2.putText(new_canvas, "Wrong", text_origin_accurate, font, font_scale, font_color, font_thickness, cv2.LINE_AA)
        # Save the output
        output_img.append(new_canvas)
        output_ans.append("No")
        print("Image ", index + 1, ": Prediction: No | Answer: ", correct_answer)
        if save_result:
            cv2.imwrite("output/" + str(index + 1) + ".jpg", new_canvas)

    index += 1

# ==================================================================================================== Step 4: Display the Results
# Get the accuracy
correct = 0
for i in range(len(output_ans)):
    if output_ans[i] == correct_ans[i]:
        correct += 1
accuracy = correct / len(output_ans) * 100
print("Accuracy: ", accuracy, "%")

# Display the output
for i in range(len(output_img)):
    cv2.imshow("Output", output_img[i])
    cv2.waitKey(0)
cv2.destroyAllWindows()

Image  1 : Prediction: Yes | Answer:  No
Image  2 : Prediction: Yes | Answer:  Yes
Image  3 : Prediction: No | Answer:  Yes
Image  4 : Prediction: Yes | Answer:  Yes
Image  5 : Prediction: No | Answer:  Yes
Image  6 : Prediction: Yes | Answer:  Yes
Image  7 : Prediction: No | Answer:  No
Image  8 : Prediction: Yes | Answer:  No
Image  9 : Prediction: No | Answer:  No
Image  10 : Prediction: Yes | Answer:  Yes
Image  11 : Prediction: Yes | Answer:  Yes
Image  12 : Prediction: Yes | Answer:  Yes
Image  13 : Prediction: Yes | Answer:  Yes
Image  14 : Prediction: No | Answer:  No
Image  15 : Prediction: No | Answer:  No
Image  16 : Prediction: No | Answer:  No
Image  17 : Prediction: Yes | Answer:  Yes
Image  18 : Prediction: Yes | Answer:  No
Image  19 : Prediction: No | Answer:  No
Image  20 : Prediction: Yes | Answer:  No
Accuracy:  70.0 %


In [4]:
# ==================================================================================================== Step 4: Classify the Tumor